In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib.pyplot as plt


from ipynb.fs.full.fonctions import boundingBoxes,resize,drawContour,checkBoxes
from ipynb.fs.full.wavelet import middle,Wavelet
from ipynb.fs.full.ransac import doRansac,plotLines
from ipynb.fs.full.white_select import selectWhite

In [6]:
%matplotlib inline
%matplotlib qt

In [7]:
# 2 frames for the substraction
frame1=None #newest frame
frame0=None #previous frame

# 1st frame will be use to detect the white borders of the field
first_frame=None
white_border=None

# index is the time of the frame
idx=0
# time jump between 2 frame
jump=100

#3D points (x,y,t)
position=[[0,0,0]]
#number of pts used for RANSAC
size_slice=15
#pts after ransac
corrected=[[0,0,0]]
#pts after correction of the trajectory
corrected2=[[0,0,0]]
# index of the last pts that had is trajectory corrected in corrected[]
lastcorrected=0 
firstLoop=True
lastLen=0

# all the lines found by RANSAC
all_lines=[ [ [[0,0,0],[0,0,0]],[[0,0,0],[0,0,0]] ] ]
#all_lines=[[[0,0,0],[0,0,0]]]

model=None
lastPosition=None


static=False
static_box=None

#templates used for the wavelet function
template = cv2.imread('template/templateRGB_Black.png')
template_HSV_W= cv2.cvtColor(template,cv2.COLOR_BGR2HSV)

templateB = cv2.imread('template/templateGoal.png')
template_HSV_B = Wavelet(templateB)

#video
cap = cv2.VideoCapture('video_record/1310.mp4')
# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")


In [8]:
#Read until video is completed
while(cap.isOpened()):
    
    # Capture frame-by-frame
    ret, frame_og= cap.read()
    
    if ret == True:
        model=None
        
        frame_og_crop=frame_og[:,:,:].copy()
        last_frame = frame_og_crop.copy()

        #detect the white border so to not confuse them with the ball
        if first_frame is None:
            frame0=frame_og_crop.copy()
            frame1=frame_og_crop.copy()
            
            first_frame = frame_og_crop.copy()
            white_border=selectWhite(first_frame)

            white_border=cv2.bitwise_not(white_border)

            display=resize(white_border)
            cv2.imshow('not border', display)

    
        frame0 = frame1.copy()
        frame1 = frame_og_crop.copy()


        maskW1=selectWhite(frame1)
        maskW0=selectWhite(frame0)

        maskW=cv2.bitwise_or(maskW1, maskW0, mask = None)
        maskW=cv2.bitwise_and(maskW,white_border, mask=None)
        maskW = cv2.erode(maskW, None, iterations=6)
        maskW = cv2.dilate(maskW, None, iterations=10)
        
        
        frame_white=cv2.bitwise_and(frame_og_crop.copy(),frame_og_crop.copy(), mask=maskW)
        display=resize(maskW)
        cv2.imshow('white select 2 frames', display)
        
        frame1RGB=cv2.cvtColor(frame1,cv2.COLOR_YUV2BGR)
        
        #draw contour on frame
        dil,contours=drawContour(maskW)
        #hide non-contoured area
        masked1=cv2.bitwise_and(frame1RGB, frame1RGB, mask=dil)
        #detection
        _,boundRect=boundingBoxes(masked1,contours)

        _, box = checkBoxes(frame_og_crop.copy(),boundRect,coeff=100,wave_temp=template_HSV_W,wave_temp2=template_HSV_B)
        
        # 3D points
        if box is not None:
            pts=middle(box)
            # if same point, don't take it
            if not np.array_equal(pts, position[-1][:2]) and ((pts[0]-position[-1][0])**2+(pts[1]-position[-1][1])**2)>100:
                position=np.append(position,[[pts[0],pts[1],idx]],axis=0) 
                idx+=50

            cv2.rectangle(frame_og_crop, (int(box[0]), int(box[1])),(int(box[0]+box[2]), int(box[1]+box[3])), (255,255,255), 2) 

            display=resize(frame_og_crop)
            cv2.imshow('detection', display)


        #Ransac
        Len=len(position)
        if Len%size_slice==0 and lastLen!=Len:
            if firstLoop:
                position=position[1:]

            sliced=position[(Len-size_slice):]

            if model is not None:
                sliced=np.concatenate((corrected[len(corrected)-2:],sliced))

            #model,droite=doRansac(sliced,frame_og_crop,distMin=25,D3=True)
            model,droite=doRansac(sliced,frame_og_crop,distMin=15,D3=True)
            lastLen=Len

            if model is not None:
                corrected=np.concatenate((corrected,model))
                all_lines=np.append(all_lines,[droite],axis=0)

                if firstLoop:
                    position=position[1:]
                    corrected=corrected[1:]
        
        
        k=lastcorrected
        lenCorrect=len(corrected)
        
        while k < (lenCorrect-5) :
            diff1=cv2.absdiff(corrected[k],corrected[k+1])
            diff2=cv2.absdiff(corrected[k],corrected[k+5])
            corrected2=np.append(corrected2,[corrected[k]],axis=0)

            if diff1[0]+diff1[1]*1 < (diff2[0]+diff2[1]):
                k+=1
            else:
                k+=4
        if firstLoop and model is not None:
            corrected2=corrected2[1:]
            firstLoop=False
        if lenCorrect>5:
            lastcorrected = lenCorrect-5


        #plot
        for index, item in enumerate(position): 
            cv2.circle(frame_og_crop, item[:2], 2, [20, 255, 20], 5)

        plotLines(corrected,lines2=corrected2,frame=frame_og_crop)

        display=resize(frame_og_crop)
        cv2.imshow('detection', display)
    
        #press key to go the next frame or Q to exit
        key = cv2.waitKey(0)
        
        if key == ord('q') or key == ord('Q'):
            break
        elif key == ord('s') or key == ord('S'):
            cv2.imwrite("image.png", frame_og)
            break
        else:
            continue
            
  # Break the loop
    else: 
        break
    

all_lines=all_lines[1:]



error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - mat is not a numpy array, neither a scalar
>  - Expected Ptr<cv::cuda::GpuMat> for argument 'mat'
>  - Expected Ptr<cv::UMat> for argument 'mat'


In [ ]:

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(111, projection='3d')
ax.grid()

ax.plot3D(corrected[:,1],corrected[:,0],corrected[:,2],'r')
ax.scatter(position[:,1],position[:,0],position[:,2],marker="+")

for item in all_lines:
     start_pts=item[0][0]
     middle_pts=item[0][1]
     end_pts=item[1][1]
     
     ax.scatter(middle_pts[1],middle_pts[0],middle_pts[2],color='k')
     ax.scatter(end_pts[1],end_pts[0],end_pts[2],color='g')
     ax.scatter(start_pts[1],start_pts[0],start_pts[2],color='g')
     
     ax.plot([start_pts[1], middle_pts[1]], [start_pts[0], middle_pts[0]], zs=[start_pts[2], middle_pts[2]],color='k')
     ax.plot([end_pts[1], middle_pts[1]], [end_pts[0], middle_pts[0]], zs=[end_pts[2], middle_pts[2]],color='k')


ax.set_ylabel('x', labelpad=20)
ax.set_ylim(0, 1500)
ax.set_xlabel('y', labelpad=20)
ax.set_xlim(0, 1500)
ax.set_zlabel('t', labelpad=20)
#ax.set_zlim(0, 100)

fig = plt.figure(figsize = (8,8))

plt.plot(corrected[:,2],corrected[:,1],'g')
plt.plot(corrected[:,2],corrected[:,0],'r')
plt.plot(corrected[:,2],corrected[:,1],'g+')
plt.plot(corrected[:,2],corrected[:,0],'r+')

fig = plt.figure(figsize = (8,8))

plt.plot(corrected2[:,2],corrected2[:,1],'k')
plt.plot(corrected2[:,2],corrected2[:,0],'b')

plt.show()

plotLines(corrected,corrected2,frame_og_crop)

key = cv2.waitKey(0)
#When everything done, release the video capture object
cap.release()  
# Closes all the frames
cv2.destroyAllWindows()

In [ ]:

template = cv2.imread('template/templateGoal.png')
template_HSV= cv2.cvtColor(template,cv2.COLOR_BGR2HSV)
M=template_HSV.shape[0]//2
middle_pts=template_HSV[M,M,:]
print(middle_pts)

wave_temp2 = Wavelet(templateB)


[ 90   7 255]
